# Lab 3: Search Terms with Pandas Part B
### Created By: Christian Basso
___

This notebook is a continuation of Lab 3. It will explore lab 3's algorithims on a differnet data set. Note that after the data import, the code is exactly the same as lab 3 minus the annotaions.

The data set that this notebook focuses on is over 200,000 jepordy questions that have been asked in the games history. This dataset is from reddit user _trexmatt_. The csv file contians dates, show numbers, questions, and answers. However, for this analysis only the answer column will be addressed.

Before running this notebook one can hypothosize that each cleaning functions will take one fifth the time when compared to the direct supply search data set. This estiamte comes from the size of the dataset, the direct supply dataset has aprox. five times the amount of lines than the jeopardy dataset. Additionally, both cleaning functions are O(n), so the time does not need to be multiplied by the time complexity. 
In regard to the dictionary lambda however, the jeopardy dataset will take 1/25 the time since the spellcheck algorithims take O(n^2) to complete.

In [1]:
import pandas as pd
import sys
from spellchecker import SpellChecker

ModuleNotFoundError: No module named 'spellchecker'

To remove the unwanted data from this set, the csv will be put directly into a dataframe, the answer column will be extracted, and put into a python list. However, this dataset contains floats and ints, so the map function is used to convert all values to strings. This will put it in the correct format for the algorithims to work.

In [12]:
df = pd.read_csv("data/JEOPARDY_CSV.csv")
searches = df[" Answer"].values.tolist()
searches = list(map(str, searches))

['Copernicus', 'Jim Thorpe', 'Arizona', "McDonald's", 'John Adams', 'the ant', 'the Appian Way', 'Michael Jordan', 'Washington', 'Crate & Barrel', 'Jackie Gleason', 'the cud', 'Ceylon (or Sri Lanka)', 'Jim Brown', 'the UV index']


_All code after this line is from lab 3_
___

In [13]:
split_searches = []
for i in searches:
    split_searches.append(i.replace("%20", " ").split(" "))


In [14]:
df = pd.DataFrame( {"Searches" : split_searches})

,Searches
0,[Copernicus]
1,"[Jim, Thorpe]"
2,[Arizona]
3,[McDonald's]
4,"[John, Adams]"
5,"[the, ant]"
6,"[the, Appian, Way]"
7,"[Michael, Jordan]"
8,[Washington]
9,"[Crate, &, Barrel]"


In [15]:
column = df["Searches"]

In [16]:
def clean_char(token):
    punct_searches = []
    remove = [".", ",", "/", "?", "!", "$", "@", "&", "*", "(", ")", "#", "%", "^", "-", "_", "\"", "'", "[", "]"]
    for j in token:
        temp = j
        for k in remove:
            temp = temp.replace(k, "")
        if temp != '':
            punct_searches.append(temp)

    return punct_searches

In [17]:
def clean_num(token):
    num_searches = []
    remove = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for j in token:
        temp = j
        for k in remove:
            temp = temp.replace(k, "")
        if temp != '':
            num_searches.append(temp)

    return num_searches

In [18]:
%%time
cleaned_chars = column.map(lambda search : clean_char(search))

Wall time: 821 ms


In [19]:
%%time
cleaned_searches = cleaned_chars.map(lambda search : clean_num(search))

Wall time: 490 ms


In [20]:
df["Searches"] = cleaned_searches

,Searches
0,[Copernicus]
1,"[Jim, Thorpe]"
2,[Arizona]
3,[McDonalds]
4,"[John, Adams]"
5,"[the, ant]"
6,"[the, Appian, Way]"
7,"[Michael, Jordan]"
8,[Washington]
9,"[Crate, Barrel]"


In [21]:
%%time
df = df[df['Searches'].map(lambda d: len(d)) > 0]

Wall time: 75.5 ms


In [23]:
def make_spelling_dict(clean_list):
    clean_dic = {"misspelled" : "correct"}
    spell = SpellChecker(distance = 1)
    for i in clean_list:
        for j in i:
            correct = spell.correction(j)
            clean_dic[j] = correct
    return clean_dic

diction = make_spelling_dict(df['Searches'])

NameError: name 'SpellChecker' is not defined

In [24]:
def compare_to_dict(token, dicton):
    l = []
    for i in token:
        l.append(diction[str(i)])
    return l

In [25]:
%%time
df = df[df['Searches'].map(lambda d : compare_to_dict(d, diction))]

NameError: name 'diction' is not defined

In [26]:
data_list = df["Searches"].values.tolist()

[['Copernicus'], ['Jim', 'Thorpe'], ['Arizona'], ['McDonalds'], ['John', 'Adams'], ['the', 'ant'], ['the', 'Appian', 'Way'], ['Michael', 'Jordan'], ['Washington'], ['Crate', 'Barrel'], ['Jackie', 'Gleason'], ['the', 'cud'], ['Ceylon', 'or', 'Sri', 'Lanka'], ['Jim', 'Brown'], ['the', 'UV', 'index']]


In [27]:
def count_frequency(l):
    freq = {}
    for item in l:
        for k in item:
            if (k in freq):
                freq[k] += 1
            else:
                freq[k] = 1

    str_builder = ""
    for key, value in freq.items():
        t = str(key) + " , " + str(value)
        print(t)
        str_builder += t + "\n"

    return str_builder

In [28]:
%%time
method_A_freq = count_frequency(data_list)
print(method_A_freq)

Copernicus , 26
Jim , 192
Thorpe , 22
Arizona , 133
McDonalds , 39
John , 2038
Adams , 250
the , 19179
ant , 13
Appian , 11
Way , 81
Michael , 368
Jordan , 138
Washington , 554
Crate , 1
Barrel , 12
Jackie , 96
Gleason , 11
cud , 2
Ceylon , 8
or , 1381
Sri , 49
Lanka , 50
Brown , 261
UV , 5
index , 12
Bulova , 1
Jesse , 85
James , 864
imp , 1
International , 82
Lou , 78
Gehrig , 25
Morocco , 63
Paul , 585
Bonwit , 1
Hattie , 9
McDaniel , 8
for , 409
her , 50
role , 2
in , 1021
Gone , 64
with , 210
Wind , 117
era , 9
Congress , 65
Party , 149
Wilt , 18
Chamberlain , 40
K , 82
Ethan , 31
Allen , 150
ply , 2
Horton , 17
Nixon , 215
a , 11313
kennel , 5
Moses , 109
Aerosmith , 14
oratory , 2
Coolidge , 65
Chester , 31
Arthur , 226
business , 17
class , 16
Muhammed , 2
Mystery , 34
Train , 42
an , 1490
oldfashioned , 2
Yertle , 6
Air , 126
France , 241
Quakers , 33
Sheena , 10
Easton , 10
organza , 1
Mulberry , 8
Street , 226
Ted , 98
Knox , 36
The , 7256
Clash , 11
XC , 2
Bartholomew , 5
C

In [ ]:
%%time
method_B_freq = df['Searches'].value_counts()
print(method_B_freq)

In [28]:
def get_mem_list(l):
    total = 0
    for i in l:
        total = total + sys.getsizeof(i)
    return total


list_mem = get_mem_list(data_list)
df_mem = df.memory_usage(deep=True)

print("Memory size of list is " + str(list_mem / 1000) + " kb")
print("Memory size of dataframe is " + str(df_mem / 1000) + " kb")

Memory size of list is 78596.848 kb
Memory size of dataframe is Index        7144.848
Searches    71452.000
dtype: float64 kb


# conclusion